In [ ]:
# 1.1 更新套件
!sudo apt-get update

In [ ]:
# 1.2 安裝 Azure CLI
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

In [ ]:
# 1.3 登入 Azure
!az login

In [ ]:
# 1.4 相關資訊設定
# 設定資源群組所在位置
resource_group_location = "westus"

# 設定資源群組名稱
date = "$(date -d '8hour' +'%Y%m%d')"
resource_group = "storage-group-" + date

# 設定儲存帳號
storage_account = "test" + date

In [ ]:
# 1.5 建立資源群組
!az group create --name $resource_group --location $resource_group_location

In [ ]:
# 1.6 建立儲存帳號
!az storage account create -n $storage_account -g $resource_group

In [ ]:
# 1.7 安裝 azure-storage-blob 套件
!pip install azure-storage-blob

In [ ]:
# 1.8 取得 Azure Storage Account 的 Connection String
conn_str = !az storage account show-connection-string -n $storage_account -g $resource_group | grep -o '"connectionString": "[^"]*' | grep -o '[^"]*$'

In [ ]:
# 1.9 設定 container_name 與 blob_name
container_name = "mycontainer"
blob_name = "SampleSource.txt"

In [ ]:
# 1.10 產生 SampleSource.txt 檔案
!echo "hello world" > $blob_name

In [ ]:
# 1.11 匯入套件
import os
from azure.storage.blob import ContainerClient, BlobClient

In [ ]:
# 1.12 建立 container
container_client = ContainerClient.from_connection_string(conn_str=conn_str[0], container_name=container_name)
container_client.create_container()

In [ ]:
# 1.13 上傳檔案
blob = BlobClient.from_connection_string(conn_str=conn_str[0], container_name=container_name, blob_name='images/'+blob_name)

with open("./" + blob_name, "rb") as data:
  blob.upload_blob(data)

In [ ]:
# 1.14 列出 container 中的 blob
container = ContainerClient.from_connection_string(conn_str=conn_str[0], container_name=container_name)

blob_list = container.list_blobs()
for blob in blob_list:
  print(blob.name)

In [ ]:
# 1.15 下載 container 中的 blob
blob = BlobClient.from_connection_string(conn_str=conn_str[0], container_name=container_name, blob_name='images/'+blob_name)
fileName, fileExtension = os.path.splitext(blob_name)

with open("./" + fileName + "-download" + fileExtension, "wb") as my_blob:
    blob_data = blob.download_blob()
    blob_data.readinto(my_blob)

In [ ]:
# 1.16 查看是否成功下載
!ls | grep "SampleSource-download.txt"

In [ ]:
# 1.17 刪除 blob
blob = BlobClient.from_connection_string(conn_str=conn_str[0], container_name=container_name, blob_name='images/'+blob_name)
blob.delete_blob()

In [ ]:
# 1.18 刪除 container
container_client = ContainerClient.from_connection_string(conn_str=conn_str[0], container_name=container_name)
container_client.delete_container() 

In [ ]:
# 1.19 刪除 Storage Account
!az storage account delete --y -n $storage_account -g $resource_group